In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [2]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [3]:
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

In [4]:
df_list = []
for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-06-28  187.929993  189.899994  187.600006  189.250000  188.994781   
       2023-06-29  189.080002  190.070007  188.940002  189.589996  189.334320   
       2023-06-30  191.630005  194.479996  191.259995  193.970001  193.708420   
       2023-07-03  193.779999  193.880005  191.759995  192.460007  192.200470   
       2023-07-05  191.570007  192.979996  190.619995  191.330002  191.071976   

                     Volume  
Ticker Date                  
AAPL   2023-06-28  51216800  
       2023-06-29  46347300  
       2023-06-30  85069600  
       2023-07-03  31458200  
       2023-07-05  46920300  


In [6]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-06-28  187.929993  189.899994  187.600006  189.250000   
1   AAPL 2023-06-29  189.080002  190.070007  188.940002  189.589996   
2   AAPL 2023-06-30  191.630005  194.479996  191.259995  193.970001   
3   AAPL 2023-07-03  193.779999  193.880005  191.759995  192.460007   
4   AAPL 2023-07-05  191.570007  192.979996  190.619995  191.330002   

    Adj Close    Volume  
0  188.994781  51216800  
1  189.334320  46347300  
2  193.708420  85069600  
3  192.200470  31458200  
4  191.071976  46920300  


In [7]:
#  performance in the stock market of all the companies:
fig=px.line(df,x='Date',y='Close',color='Ticker',title='Stock Market Performance for the Last 3 Months')
fig

In [8]:
#faceted area chart to compare performance of different companies
fig=px.area(df,x='Date',y='Close',color='Ticker',facet_col='Ticker',labels={'Date':'Date','Close':'Closing Price','Ticker':'Company'})
fig.show()

In [9]:
df['MA10']=df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0,drop=True)
df['MA20']=df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0,drop=True)
df_grouped=df.groupby('Ticker')
for ticker1, group in df_grouped:
    print(f'Moving Averages for {ticker1}')
    print(group[['MA10' ,'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
60  176.187001  179.906000
61  175.859001  179.779500
62  175.425002  179.368000
63  175.047000  178.683500
64  174.567009  177.848005

[65 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
195         NaN         NaN
196         NaN         NaN
197         NaN         NaN
198         NaN         NaN
199         NaN         NaN
..          ...         ...
255  136.359003  135.906001
256  135.802002  135.980001
257  135.140001  135.863001
258  134.536002  135.661501
259  133.941000  135.467001

[65 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
69          NaN         NaN
..          ...         ...
125  328.970999  329.03

In [16]:
#visualize moving averages of all companies
for ticker,group in df_grouped:
    fig=px.line(group,x='Date',y=['Close','MA10','MA20'],title=f"{ticker} Moving Averages")
    fig.show()

In [11]:
#Analyze the Votality of all companies
df['Votality']=df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0,drop=True)
df_group=df.groupby('Ticker')
for t ,group1 in df_group:
    print(f'Votality of {t}')
    print(group1['Votality'])

Votality of AAPL
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
        ...   
60    0.012306
61    0.012367
62    0.013359
63    0.013158
64    0.012747
Name: Votality, Length: 65, dtype: float64
Votality of GOOG
195         NaN
196         NaN
197         NaN
198         NaN
199         NaN
         ...   
255    0.013985
256    0.014219
257    0.014984
258    0.015627
259    0.015759
Name: Votality, Length: 65, dtype: float64
Votality of MSFT
65          NaN
66          NaN
67          NaN
68          NaN
69          NaN
         ...   
125    0.013764
126    0.012831
127    0.012705
128    0.011293
129    0.010639
Name: Votality, Length: 65, dtype: float64
Votality of NFLX
130         NaN
131         NaN
132         NaN
133         NaN
134         NaN
         ...   
190    0.017430
191    0.018529
192    0.018222
193    0.012399
194    0.010474
Name: Votality, Length: 65, dtype: float64


In [12]:
fig=px.line(df,x='Date',y='Votality',color='Ticker',title='Votality of All companies')
fig.show()

In [13]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple=df.loc[df['Ticker']=='AAPL',['Date','Close']].rename(columns={'Close':'AAPL'})
microsoft=df.loc[df['Ticker']=='MSFT',['Date','Close']].rename(columns={'Close':'MSFT'})
df_corr=pd.merge(apple,microsoft,on='Date')
df_corr
# create a scatter plot to visualize the correlation
fig=px.scatter(df_corr,x='AAPL',y='MSFT',trendline='ols',title='Correlation between Apple and Microsoft')
fig.show()

In [15]:
for a in df:

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
    fig.show()